# Batch download of NRRD from XNAT collection

In [1]:
import os, subprocess, glob
import glob
import shutil
import xnat #needed for reading and writing to/from XNAT

In [2]:
def cleanup_temp_folder(dirt):
    if os.path.exists(dirt):
        shutil.rmtree(dirt)
        os.makedirs(dirt)
    if not os.path.exists(dirt):
        os.makedirs(dirt)

In [3]:
#------------------------- USER SETTINGS -------------------------------

# set up XNAT login credentials here
# .......................
xnatUrl = 'http://xnat-nginx:80'  #change me! e.g. 'http://localhost:8081/'
# - check the url using docker network inspect on the package defaul network, find the web application nginx
# - the procedure is
# - docker network ls
# - find your docker network bridge running all the xnat and python images, eg xnat-docker-compose-master_default
# - copy the network id on the left hand side
# - docker network inspect [the network id from above]
# - scroll down a little and find the docker IP network address of the nginx gateway
#
#xnatUrl = 'https://xnat.bmia.nl'
xnatUser = 'admin'  #change me!
#xnatUser = 'leonardwee'
xnatPass = 'admin'  #change me!
xnatProject = 'Test_01'  #change me!


### main section

In [4]:
PyradiomicsWorkingDir = './pre-process-RADIOMICS'
PyradiomicsSetLabel = 'pyr_'

In [5]:
#CAREFUL - empties the following before filling it up from XNAT
cleanup_temp_folder(PyradiomicsWorkingDir)

In [6]:
# -----------------------------------------------------------
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject = session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
            mySubjectID = s.label
            mySubject = myProject.subjects[mySubjectID]
            myExperimentsList = mySubject.experiments.values()
            for e in myExperimentsList:
                myExperimentID = e.label
                myExperiment = mySubject.experiments[myExperimentID]
                myResourcesList = myExperiment.resources.values()
                for r in myResourcesList:
                    myResourceID = r.label
                    if myResourceID.startswith(PyradiomicsSetLabel):
                        myResource = myExperiment.resources[myResourceID]
                        for i in range(len(myResource.files)):
                            myFile = myResource.files[i].data['Name']
                            if myFile.startswith('GTV'):
                                myStem = mySubjectID + '_' + myResourceID + '.csv'
                                try:
                                    myResource.files[0].download(os.path.join(PyradiomicsWorkingDir,myStem))
                                except:
                                    print('FAILURE - could not save ' + os.path.join(PyradiomicsWorkingDir,myStem))


[WARNING] Detected a redirect from http://xnat-nginx:80 to http://xnat-nginx/, using http://xnat-nginx/ from now on
100% of  47.3 KiB |################################|  11.9 MiB/s Time:  0:00:00
100% of  47.2 KiB |################################|   4.9 MiB/s Time:  0:00:00


In [7]:
import pandas as pd

all_files = glob.glob(PyradiomicsWorkingDir + "/*.csv")
li = []

for fname in all_files:
    #temp = fname.split('.')[1].split('/')[-1]
    #subj = temp.split('_')[0]
    #mask = temp.split('_')[-1]
    try :
        df = pd.read_csv(fname, index_col=None, header=0)
        #df.insert(0, 'Subject_label', subj)
        #df.insert(1, 'ROI_label', mask)
        li.append(df)
    except :
        print("This did not have the expected data!")

frame = pd.concat(li, axis=0, ignore_index=True)

frame.to_csv('./plastimatch_version_hn1_strategy_wavelets.csv', index=False)
               